## LangGraph

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph


### LLM

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

### Agents

In [40]:
from langchain.schema import HumanMessage

def agent_one(state):
    """First agent that processes input and adds a step."""
    messages = [
        HumanMessage(content="Summarize: " + state.input)
    ]   
    response = llm.invoke(messages)  
    state.summary = response.content
    return state

def agent_two(state):
    """Second agent that generates keywords from the summary."""
    messages = [
        HumanMessage(content="Extract keywords: " + state.summary)
    ]  
    response = llm.invoke(messages)
    state.keywords = response.content
    return state

def agent_three(state):
    """Third agent that generates a final refined response."""
    messages = [
        HumanMessage(content="Generate a short response using: " + state.keywords)
    ]
    response = llm.invoke(messages)
    state.final_response = response.content
    return state

### Workflow (Nodes)

In [41]:
from pydantic import BaseModel

class State(BaseModel):
    input: str
    summary: str | None = None
    keywords: str | None = None
    final_response: str | None = None

In [42]:
graph = StateGraph(State)
graph.add_node("AgentOne", agent_one)
graph.add_node("AgentTwo", agent_two)
graph.add_node("AgentThree", agent_three)

### Edges

In [43]:
graph.add_edge("AgentOne", "AgentTwo")
graph.add_edge("AgentTwo", "AgentThree")

graph.set_entry_point("AgentOne")

### Run the Graph

In [46]:
runnable_graph = graph.compile()

initial_state = State(input="LangChain simplifies AI workflows.")
final_state = State(**runnable_graph.invoke(initial_state))

print(final_state.summary)
print(final_state.keywords)
print(final_state.final_response)


LangChain streamlines the process of working with artificial intelligence by making it easier and more efficient.
Keywords: LangChain, streamlines, process, working, artificial intelligence, easier, efficient.
LangChain streamlines the process of working with artificial intelligence, making it easier and more efficient for users.
